In [42]:
import torch

torch.cuda.empty_cache()

In [1]:
import pandas as pd


captions_file = "../data/10k_blip_captions.csv"
mapped_question_answers_file = "../data/10k_mapped_question_answers.csv"

captions_data = pd.read_csv(captions_file)
qa_data = pd.read_csv(mapped_question_answers_file)


qa_data.drop(columns=["Image file"], inplace=True)
data = pd.concat([qa_data, captions_data], axis=1)

tmp_data= data[:3]
tmp_data
# llamaPipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf", device='cuda')

# Define a function to generate text
# def generate_text(row):
#     prompt = f"Based on the image caption (generated by BLIP model): {row['Generated Caption']}, Answer the question '{row['Question']}' \n Answer: "
#     generated_text = llamaPipe(prompt, max_new_tokens=20)[0]['generated_text']
#     return generated_text

# Apply the function to each row of the dataframe
# tmp_data['Generated Answer'] = tmp_data.apply(generate_text, axis=1)

# # Write data to a CSV file
# tmp_data.to_csv('tmp.csv', index=False)

,Image ID,Question ID,Question,Answer,Plausible answers,Question Type,Answer Type,Image file,Generated Caption
0,62549,62549016,Is the bird a carnivore?,yes,"{'no', 'yes'}",is the,yes/no,../data/validation/images/COCO_val2014_0000000...,a man holding a bird
1,46870,46870003,What kind of bike is it?,honda,"{'honda', 'motorcycle', 'honda motorcycle'}",what kind of,other,../data/validation/images/COCO_val2014_0000000...,a motorcycle parked on the street
2,29080,29080002,Is the man smiling?,yes,"{'no', 'yes'}",is the man,yes/no,../data/validation/images/COCO_val2014_0000000...,a man sitting on the ground with a large object


In [4]:
answers = [{
    "Test": instance
    # 'Image ID': instance['Image ID']
} for instance in tmp_data]

In [7]:

list_of_dicts

[{'Image ID': 62549,
  'Question ID': 62549016,
  'Question': 'Is the bird a carnivore?',
  'Answer': 'yes',
  'Plausible answers': "{'no', 'yes'}",
  'Question Type': 'is the',
  'Answer Type': 'yes/no',
  'Image file': '../data/validation/images/COCO_val2014_000000062549.jpg',
  'Generated Caption': 'a man holding a bird'},
 {'Image ID': 46870,
  'Question ID': 46870003,
  'Question': 'What kind of bike is it?',
  'Answer': 'honda',
  'Plausible answers': "{'honda', 'motorcycle', 'honda motorcycle'}",
  'Question Type': 'what kind of',
  'Answer Type': 'other',
  'Image file': '../data/validation/images/COCO_val2014_000000046870.jpg',
  'Generated Caption': 'a motorcycle parked on the street'},
 {'Image ID': 29080,
  'Question ID': 29080002,
  'Question': 'Is the man smiling?',
  'Answer': 'yes',
  'Plausible answers': "{'no', 'yes'}",
  'Question Type': 'is the man',
  'Answer Type': 'yes/no',
  'Image file': '../data/validation/images/COCO_val2014_000000029080.jpg',
  'Generated Ca

In [33]:
# # Initialize Llama pipeline
# llamaPipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf", device='cuda')

# # Define a function to generate text
# def generate_text(row):
#     prompt = f"Based on the image caption (generated by BLIP model): {row['Caption']}, Answer the question '{row['Question']}' \n Answer: "
#     generated_text = llamaPipe(prompt, max_new_tokens=20)[0]['generated_text']
#     return generated_text

# # Apply the function to each row of the dataframe
# df['Generated Answer'] = df.apply(generate_text, axis=1)

# # Write data to a CSV file
# df.to_csv('10k_default_answers.csv', index=False)

tmp_data.loc[:, "Prompt"] = "Based on the image caption (generated by BLIP model): " + tmp_data['Generated Caption'] +", Answer the question " + tmp_data['Question'] +" '' \n Answer: "
tmp_data

,Image ID,Question ID,Question,Answer,Plausible answers,Question Type,Answer Type,Image file,Generated Caption,Prompt
0,62549,62549016,Is the bird a carnivore?,yes,"{'no', 'yes'}",is the,yes/no,../data/validation/images/COCO_val2014_0000000...,a man holding a bird,Based on the image caption (generated by BLIP ...
1,46870,46870003,What kind of bike is it?,honda,"{'honda', 'motorcycle', 'honda motorcycle'}",what kind of,other,../data/validation/images/COCO_val2014_0000000...,a motorcycle parked on the street,Based on the image caption (generated by BLIP ...
2,29080,29080002,Is the man smiling?,yes,"{'no', 'yes'}",is the man,yes/no,../data/validation/images/COCO_val2014_0000000...,a man sitting on the ground with a large object,Based on the image caption (generated by BLIP ...


In [35]:
# Initialize Llama pipeline
from transformers import pipeline
llamaPipe = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf", device='cuda')

# Define a function to generate text
def generate_text(input_data):
    generated_text = llamaPipe(input_data, max_new_tokens=20)[0]['generated_text']
    return generated_text

# Apply the function to column 'A' of the dataframe
tmp_data.loc[:, 'Generated Answer'] = tmp_data['Prompt'].apply(generate_text)

/home/samavedam.m/.conda/envs/llm_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.21s/it]
/tmp/ipykernel_20346/1825709259.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_data.loc[:, 'Generated Answer'] = tmp_data['Prompt'].apply(generate_text)
